<a href="https://colab.research.google.com/github/JoungMinJu/lab_study/blob/main/RNN%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_%ED%85%8D%EC%8A%A4%ED%8A%B8_%EC%83%9D%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

다대일 구조의 RNN을 사용해 문맥 반영해서 텍스트 생성하는 모델을 만들기


In [1]:
#데이터에 대한 이해와 전처리
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

In [2]:
text="""경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""
#3개의 한국어 문장 저장


In [3]:
#단어 집합을 생성하고 크기를 확인해보겠습니다.

t = Tokenizer()
t.fit_on_texts([text])
vocab_size = len(t.word_index) + 1

# 케라스 토크나이저의 정수 인코딩은 인덱스가 1부터 시작하지만,
# 케라스 원-핫 인코딩에서 배열의 인덱스가 0부터 시작하기 때문에
# 배열의 크기를 실제 단어 집합의 크기보다 +1로 생성해야하므로 미리 +1 선언 
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 12


In [4]:
#각 단어와 단어에 부여된 정수 인덱스 출력
print(t.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [8]:
#훈련 데이터 만들기
sequences = list()
for line in text.split('\n'): # Wn을 기준으로 문장 토큰화
    #내가 궁금해서
    print("line:",line)
    encoded = t.texts_to_sequences([line])[0]
    #내가 궁금해서
    print("encoded:",encoded)
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        #내가 궁금해서
        print("sequences:",sequence)
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))

line: 경마장에 있는 말이 뛰고 있다
encoded: [2, 3, 1, 4, 5]
sequences: [2, 3]
sequences: [2, 3, 1]
sequences: [2, 3, 1, 4]
sequences: [2, 3, 1, 4, 5]
line: 
encoded: []
line: 그의 말이 법이다
encoded: [6, 1, 7]
sequences: [6, 1]
sequences: [6, 1, 7]
line: 
encoded: []
line: 가는 말이 고와야 오는 말이 곱다
encoded: [8, 1, 9, 10, 1, 11]
sequences: [8, 1]
sequences: [8, 1, 9]
sequences: [8, 1, 9, 10]
sequences: [8, 1, 9, 10, 1]
sequences: [8, 1, 9, 10, 1, 11]
line: 
encoded: []
학습에 사용할 샘플의 개수: 11


In [6]:
print(sequences)

#아직 레이블로 사용될 단어를 분리하지 않은 훈련 데이터
#[2,3]=[경마장에, 있는]
#[2,3,1]=[경마장에, 있는,말이]

#전체 훈련 데이터에 대해서 맨 우측에 있는 단어에 대해서만 레이블로 분리해야한다.



[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [9]:
#전체 샘플에 대해서 길이를 일치시켜준다. 가장 긴 샘플의 길이 기준
max_len=max(len(l) for l in sequences) # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
print('샘플의 최대 길이 : {}'.format(max_len))

sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences)

샘플의 최대 길이 : 6
[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [10]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]
# 리스트의 마지막 값을 제외하고 저장한 것은 X
# 리스트의 마지막 값만 저장한 것은 y. 이는 레이블에 해당됨.

In [11]:
print(X)
print(y)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]
[ 3  1  4  5  1  7  1  9 10  1 11]


In [12]:
#데이터 훈련시키기 전에 레이블에 대해서 원-핫 인코딩을 수행
y = to_categorical(y, num_classes=vocab_size)
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [14]:
#모델 설계하기

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [15]:
model=Sequential()

In [16]:
model.add(Embedding(vocab_size, 10, input_length=max_len-1))#레이블을 분리하였으므로1 빼준다.
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)


#각 단어의 임베딩 벡터는 10차원을 가지고, 32의 은닉상태크기를 가지는  바닐라 RNN을 사용

Epoch 1/200
1/1 - 1s - loss: 2.5036 - accuracy: 0.0909
Epoch 2/200
1/1 - 0s - loss: 2.4885 - accuracy: 0.0909
Epoch 3/200
1/1 - 0s - loss: 2.4739 - accuracy: 0.0909
Epoch 4/200
1/1 - 0s - loss: 2.4597 - accuracy: 0.0909
Epoch 5/200
1/1 - 0s - loss: 2.4457 - accuracy: 0.0909
Epoch 6/200
1/1 - 0s - loss: 2.4317 - accuracy: 0.1818
Epoch 7/200
1/1 - 0s - loss: 2.4176 - accuracy: 0.1818
Epoch 8/200
1/1 - 0s - loss: 2.4034 - accuracy: 0.3636
Epoch 9/200
1/1 - 0s - loss: 2.3888 - accuracy: 0.3636
Epoch 10/200
1/1 - 0s - loss: 2.3738 - accuracy: 0.4545
Epoch 11/200
1/1 - 0s - loss: 2.3583 - accuracy: 0.5455
Epoch 12/200
1/1 - 0s - loss: 2.3422 - accuracy: 0.3636
Epoch 13/200
1/1 - 0s - loss: 2.3254 - accuracy: 0.3636
Epoch 14/200
1/1 - 0s - loss: 2.3078 - accuracy: 0.3636
Epoch 15/200
1/1 - 0s - loss: 2.2894 - accuracy: 0.3636
Epoch 16/200
1/1 - 0s - loss: 2.2700 - accuracy: 0.3636
Epoch 17/200
1/1 - 0s - loss: 2.2497 - accuracy: 0.3636
Epoch 18/200
1/1 - 0s - loss: 2.2284 - accuracy: 0.3636
E

In [17]:
#모델이 정확하게 예측하고 있는지 문장을 생성하는 함수를 만들어 출력해보자


def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=5, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
    
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [19]:
print(sentence_generation(model, t, '경마장에', 4))

#경마장에라는 단어 뒤에는 총 네개의 단어가 있으므로 네번 예측

경마장에 있는 말이 뛰고 있다


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [20]:
print(sentence_generation(model, t, '그의', 2)) # 2번 예측

그의 말이 법이다


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [21]:
print(sentence_generation(model, t, '가는', 5)) # 5번 예측


# 충분한 훈련 데이터가 없으므로 문장의 길이에 맞게 적절하게 예측해야하는 횟수 4,2, 5를 지정해줘야함


가는 말이 고와야 오는 말이 곱다


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
